# 1.6 Intro to NLP and Network Analysis

1. Install the necessary libraries from the requirements.txt file from this Exercise in your corresponding virtual environment.
2. Start a new notebook in JupyterLab and import the libraries you’ll need.

In [2]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

In [3]:
# Download English module

!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

## Load 20th Centuy Wiki page

3. Load the twentieth-century text file.

In [6]:
# Load wiki page

with open('20th_Century_wiki.txt', 'r', errors='ignore') as file:
    data = file.read().replace('\n', '')

4. There are no corrections needed for the countries list because the countries do not have alternate names in the text. 

5. Use the text file to create a NER object.

In [7]:
book = NER(data)

In [8]:
# Visualize identified entities

displacy.render(book[273:20000], style = "ent", jupyter = True)

## Get named entity list per sentence

6. Split the sentence entities from the NER object.

In [10]:
df_sentences = [] # empty shell to store results

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
    entity_list = [ent.text for ent in sent.ents]
    df_sentences.append({"sentence": sent, "entities": entity_list})
    
df_sentences = pd.DataFrame(df_sentences)

In [11]:
df_sentences.head(10)

sentence  \
0  (Key, events, of, the, 20th, century, -, Wikip...   
1                           (the, wars1.2.1Economic)   
2  (depression1.2.2The, rise, of, dictatorship1.3...   
3  (World, War, II, (, 1939–1945)1.3.1The, war, i...   
4  (days1.3.7The, war, in, the, Pacific1.3.7.1Bac...   
5  (decolonization1.4.2The, Cold, War, (, 1947–19...   
6  (The, World, Wars, sparked, tension, between, ...   
7  (These, advancements, have, played, a, signifi...   
8  (Historic, events, in, the, 20th, century[edit...   
9  (The, 1900s, saw, the, decade, herald, a, seri...   

                                            entities  
0  [the 20th century - WikipediaJump, Navigation\...  
1                                                 []  
2                                                 []  
3                                     [World War II]  
4  [Pacific1.3.7.1Background1.3.8Japanese, Holoca...  
5  [decolonization1.4.2The Cold War, 1947–1991)1....  
6  [the Cold War, the Space Race, the World Wide ...  
7                          [the 21st century, today]  
8            [the 20th, Edwardian, the 20th century]  
9                            [The 1900s, the decade]

## Load countries

In [13]:
# Import countries

countries_df = pd.read_csv("countries_list_20th_century_1.5.csv", index_col = 0)

In [14]:
countries_df.head()

country_name
1   Afghanistan 
2       Albania 
3       Algeria 
4       Andorra 
5        Angola

In [15]:
countries_df["country_name"] = countries_df["country_name"].apply(lambda x: x.strip(" "))

## Filtering entities from the csv

7. Filter the entities so that you end up only with the ones from your countries list.

In [17]:
# Function to filter out entities not of interest

def filter_entity(ent_list, countries_df):
    return [ent for ent in ent_list 
            if ent in list(countries_df['country_name'])]

In [18]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, countries_df))

In [19]:
df_sentences['country_entities'].head(20)

0                              []
1                              []
2                              []
3                              []
4                              []
5                              []
6                              []
7                              []
8                              []
9                              []
10                             []
11                             []
12                             []
13                             []
14                             []
15                             []
16                             []
17        [France, Italy, Russia]
18    [Germany, Bulgaria, Russia]
19              [Germany, Russia]
Name: country_entities, dtype: object

In [20]:
# Filter out sentences that don't have any country entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

In [21]:
df_sentences_filtered

sentence  \
17    (The, Allies, ,, known, initially, as, ", The,...   
18    (Germany, ,, Austria, -, Hungary, ,, Bulgaria,...   
19    (The, Bolsheviks, negotiated, the, Treaty, of,...   
20    (In, the, treaty, ,, Bolshevik, Russia, ceded,...   
22    (Although, Germany, shifted, huge, forces, fro...   
...                                                 ...   
947   (", Selling, ", Operation, Passage, to, Freedo...   
970   (", Stuck, in, Endless, Preliminaries, :, Viet...   
1145  (", Anti, -, American, Behavior, in, the, Midd...   
1148  (The, Rise, of, China, and, India, :, A, New, ...   
1149               (Singapore, :, World, Scientific, .)   

                                               entities  \
17    [The Triple Entente, the British Empire, Franc...   
18    [Germany, Austria-Hungary, Bulgaria, the Ottom...   
19    [Bolsheviks, the Treaty of Brest-Litovsk, Germ...   
20    [Bolshevik Russia, Baltic, Germany, Kars Oblas...   
22                          [Germany, Allied, American]   
...                                                 ...   
947   [Thomas Dooley, the Religious Overtones of Ear...   
970   [Vietnam, the Battle of the Paris Peace Table,...   
1145          [Anti-American, the Middle East, Lebanon]   
1148                    [China, India, New Asian Drama]   
1149                      [Singapore, World Scientific]   

                 country_entities  
17        [France, Italy, Russia]  
18    [Germany, Bulgaria, Russia]  
19              [Germany, Russia]  
20                      [Germany]  
22                      [Germany]  
...                           ...  
947                     [Vietnam]  
970                     [Vietnam]  
1145                    [Lebanon]  
1148                      [India]  
1149                  [Singapore]  

[121 rows x 3 columns]

## Create relationships

8. Create the relationships dataframe.

In [23]:
# Defining relationships 

relationships = [] 

for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i+5, df_sentences_filtered.index[-1])
    char_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])
    

    char_unique = [char_list[i] for i in range(len(char_list)) 
                   if (i==0) or char_list[i] != char_list[i-1]]
    
    if len(char_unique) > 1:
        for idx, a in enumerate(char_unique[:-1]):
            b = char_unique[idx + 1]
            relationships.append({"source": a, "target": b})

In [24]:
relationship_df = pd.DataFrame(relationships)

In [25]:
relationship_df

source     target
0     France      Italy
1      Italy     Russia
2     France      Italy
3      Italy     Russia
4     Russia    Germany
..       ...        ...
612  Lebanon      India
613    India  Singapore
614    India  Singapore
615    India  Singapore
616    India  Singapore

[617 rows x 2 columns]

In [26]:
# Sort the cases with a->b and b->a

relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

source  target
0   France   Italy
1    Italy  Russia
2   France   Italy
3    Italy  Russia
4  Germany  Russia

In [27]:
# Summarize the interactions

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [28]:
relationship_df.head(10)

source   target  value
0    France    Italy     10
1     Italy   Russia      6
2   Germany   Russia     26
3  Bulgaria  Germany      6
4  Bulgaria   Russia      6
5   Germany    Italy     26
6   Austria  Germany     11
7   Germany    Spain      1
8    France   Poland     11
9    France  Germany     31

9. Save and export your dataframe.

In [63]:
relationship_df.to_csv('country_relationship.csv')